# YoloV8 Face Detection


In [4]:
# Import libraries

import os
import cv2
import copy
import wandb
import shutil
import random
import numpy as np
import pandas as pd
from ultralytics import YOLO
import matplotlib.pyplot as plt

In [8]:
# Download the checkpoints for YoloV8
import os
import urllib.request

# Define the folder and file details
folder = "checkpoints"
checkpoint_details = ["yolov8n-face.pt", "yolov8l-face.pt", "yolov8m-face.pt"]
os.makedirs(folder, exist_ok=True)  # Create the folder if it doesn't exist

# URLs corresponding to the checkpoints
urls = [
    "https://github.com/akanametov/yolov8-face/releases/download/v0.0.0/yolov8n-face.pt",
    "https://github.com/akanametov/yolov8-face/releases/download/v0.0.0/yolov8l-face.pt",
    "https://github.com/akanametov/yolov8-face/releases/download/v0.0.0/yolov8m-face.pt",
]

# Check and download the files
for url in urls:
    filename = os.path.basename(url)
    destination = os.path.join(folder, filename)
    
    if not os.path.exists(destination):  # Check if file already exists
        print(f"Downloading {filename}...")
        urllib.request.urlretrieve(url, destination)
        print(f"Downloaded to {destination}")
    else:
        print(f"{filename} already exists, skipping download.")


yolov8n-face.pt already exists, skipping download.
yolov8l-face.pt already exists, skipping download.
yolov8m-face.pt already exists, skipping download.


In [15]:
import yaml
from typing import Any

def validate_coco_yaml(yaml_path: str) -> bool:
    """
    Validates the structure of a COCO-style YAML configuration.

    Args:
        yaml_path (str): Path to the YAML file to validate.

    Returns:
        bool: True if the YAML is valid, False otherwise.
    """
    required_fields = {
        "path": str,
        "train": str,
        "val": str,
        "names": dict
    }
    
    try:
        # Load YAML file
        with open(yaml_path, "r", encoding="utf-8") as file:
            config = yaml.safe_load(file)
        
        # Validate top-level fields
        for field, field_type in required_fields.items():
            if field not in config:
                print(f"Error: Missing required field '{field}'.")
                return False
            if not isinstance(config[field], field_type):
                print(f"Error: Field '{field}' should be of type {field_type.__name__}.")
                return False
        
        # Validate 'names' dictionary
        names = config["names"]
        if not all(isinstance(key, int) and isinstance(value, str) for key, value in names.items()):
            print("Error: 'names' should be a dictionary with integer keys and string values.")
            return False
        
        # Optional fields
        if "test" in config and config["test"] is not None:
            if not isinstance(config["test"], str):
                print("Error: 'test' should be of type str if provided.")
                return False

        print("YAML is valid.")
        return True
    
    except yaml.YAMLError as e:
        print(f"Error parsing YAML file: {e}")
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False


In [16]:
yaml_path = "coco8.yaml"
ok = validate_coco_yaml(yaml_path)
print(ok)

YAML is valid.
True


In [18]:
# pretrained model used to detect faces
model = YOLO(f"{folder}/yolov8m-face.pt")

In [ ]:
model.train(
    data=yaml_path,
    epochs=1,
    batch=16,
    imgsz=640,
    name="exp",
    exist_ok=False,
    plots=True,
    save=True,
    resume=False,
)

Ultralytics 8.3.61  Python-3.11.9 torch-2.5.1+cpu CPU (11th Gen Intel Core(TM) i7-11800H 2.30GHz)
engine\trainer: task=detect, mode=train, model=checkpoints/yolov8m-face.pt, data=coco8.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=exp, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tr

100%|██████████| 6.66M/6.66M [00:00<00:00, 11.2MB/s]
Unzipping C:\Users\soura\Desktop\datasets\coco128.zip to C:\Users\soura\Desktop\datasets\coco128...: 100%|██████████| 263/263 [00:00<00:00, 1396.94file/s]

Dataset download success  (3.7s), saved to C:\Users\soura\Desktop\datasets



100%|██████████| 755k/755k [00:00<00:00, 3.76MB/s]

Overriding model.yaml nc=1 with nc=80

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 


...

train: Scanning C:\Users\soura\Desktop\datasets\coco128\labels\train2017... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<00:00, 195.08it/s]

train: New cache created: C:\Users\soura\Desktop\datasets\coco128\labels\train2017.cache



val: Scanning C:\Users\soura\Desktop\datasets\coco128\labels\train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]


Plotting labels to c:\Users\soura\Desktop\my-sdk\runs\detect\exp\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000119, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to c:\Users\soura\Desktop\my-sdk\runs\detect\exp
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/8 [00:00<?, ?it/s]